In [1]:
flag = False

In [2]:
if flag:
    print(Run)
flag = True

In [3]:
holdout = 0.60
canons  = 1
random_state = 1+21+int(100*holdout)+canons*100
num_samples = 5000
dense_layer_length = 100
table = 'iris'
if_exists = 'append'

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# 1. Load the Iris dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                   names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])

# 2. Separate features (X) and target (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# 3. Preprocess the data
# Encode the target labels into numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # 0, 1, 2 for Iris-setosa, Iris-versicolor, Iris-virginica

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=holdout, random_state=random_state)

In [5]:
len(X_train),len(y_test)

(60, 90)

In [5]:
from os import chdir
chdir('..')
import sopy as sp
from connect import conn
engine = conn()


2025-04-17 06:06:58.371374: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 06:06:58.381111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744888018.392942    5124 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744888018.396446    5124 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 06:06:58.408302: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
Run = pd.Series()

d = pd.DataFrame(X, columns = range(1,5)).describe()
ir= pd.DataFrame(X, columns = range(1,5))
cl= pd.Series(y_encoded)

sigmas   = [ d[c]['std']/4 for c in d]
means    = [ d[c]['mean'] for c in d]
sigmas2   =  [ [ 1/ir.loc[cl[cl==cl1].index, c].describe()['std']**2 for c in d ] for cl1 in cl.unique() ]

lattices = [ np.arange( d[c]['min'], d[c]['max'], d[c]['std']/4 ) for c in d ]

Run['canons'] = canons
Run['holdout'] = holdout
Run['random_state'] = random_state
Run['dense_layer_length'] = dense_layer_length

d = pd.DataFrame(X, columns = range(1,5)).describe()
ir= pd.DataFrame(X_train, columns = range(1,5))
cl= pd.Series(y_train)

In [7]:
u = sp.vector() 
for index in cl[cl==0].index:
    u += sp.vector().gaussian(a=1, ls = 4*[0], positions = ir.loc[index].values, sigmas = sigmas2[0] , lattices = lattices) 
u = u.mul(1./u.n()).fibonacci( partition = canons )
print(len(u))

2025-04-17 06:07:10.259619: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


1


In [8]:
u2 = sp.vector() 
for index in cl[cl==1].index:
    u2 += sp.vector().gaussian(a=1, ls = 4*[0], positions = ir.loc[index].values, sigmas = sigmas2[0] , lattices = lattices) 
u2 = u2.mul(1./u2.n()).fibonacci( partition = canons )
print(len(u2))

1


In [9]:
u3 = sp.vector() 
for index in cl[cl==2].index:
    u3 += sp.vector().gaussian(a=1, ls = 4*[0], positions = ir.loc[index].values, sigmas = sigmas2[0] , lattices = lattices) 
u3 = u3.mul(1./u3.n()).fibonacci( partition = canons )
print(len(u3))

1


In [10]:
y_pred = [ [np.argmax([ v.dot(sp.vector().delta(a=1, positions = x, spacings = sigmas , lattices = lattices )) for v in [u,u2,u3] ])]for x in X_test]

In [11]:
accuracy_sop = accuracy_score(y_test, y_pred)
accuracy_sop

0.7

In [12]:
##GEMINI GENERATED THIS CODE

#y_train = label_encoder.fit_transform(y_train)  # 0, 1, 2 for Iris-setosa, Iris-versicolor, Iris-virginica
#y_test = label_encoder.fit_transform(y_test)  # 0, 1, 2 for Iris-setosa, Iris-versicolor, Iris-virginica

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Define the Single Layer Dense Neural Network (Softmax Regression)
class SingleLayerNN:
    def __init__(self, input_size, output_size, learning_rate=0.01, epochs=1000):
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = np.random.rand(input_size, output_size) - 0.5  # Initialize weights randomly
        self.bias = np.zeros((1, output_size))  # Initialize bias to zeros

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def forward(self, X):
        self.z = np.dot(X, self.weights) + self.bias
        self.y_hat = self.softmax(self.z)
        return self.y_hat

    def backward(self, X, y, y_hat):
        m = X.shape[0]
        dz = y_hat - self.one_hot_encode(y)
        dw = (1/m) * np.dot(X.T, dz)
        db = (1/m) * np.sum(dz, axis=0, keepdims=True)
        return dw, db

    def one_hot_encode(self, y):
        one_hot = np.zeros((len(y), self.output_size))
        one_hot[np.arange(len(y)), y] = 1
        return one_hot

    def update_parameters(self, dw, db):
        self.weights -= self.learning_rate * dw
        self.bias -= self.learning_rate * db

    def train(self, X_train, y_train):
        for epoch in range(self.epochs):
            y_hat = self.forward(X_train)
            dw, db = self.backward(X_train, y_train, y_hat)
            self.update_parameters(dw, db)
            if (epoch + 1) % 100 == 0:
                loss = self.categorical_cross_entropy(self.one_hot_encode(y_train), y_hat)
                #print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {loss:.4f}")

    def predict(self, X_test):
        y_prob = self.forward(X_test)
        y_pred = np.argmax(y_prob, axis=1)
        return y_pred

    def categorical_cross_entropy(self, y_true, y_pred):
        # Add a small epsilon to avoid log(0)
        epsilon = 1e-15
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)
        loss = -np.mean(np.sum(y_true * np.log(y_pred_clipped), axis=1))
        return loss

# 5. Initialize and train the neural network
input_size = X_train_scaled.shape[1]  # Number of features (4)
output_size = len(np.unique(y_encoded))  # Number of classes (3)
learning_rate = 0.1
epochs = 2000

model = SingleLayerNN(input_size, output_size, learning_rate, epochs)
model.train(X_train_scaled, y_train)

# 6. Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# 7. Evaluate the model
accuracy_snn = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy_snn:.4f}")

# You can also print the classification report for more detailed metrics
#from sklearn.metrics import classification_report
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Test Accuracy: 0.9333


In [13]:
samples = [ v.sample(num_samples) for v in [u,u2,u3] ] 

KeyboardInterrupt: 

In [ ]:
Run['num_samples']= num_samples

In [ ]:
X_train_buddy = np.transpose(np.array(np.concat([samples[0],samples[1],samples[2]]),dtype= np.float64),(2,0,1))[0]
y_train_buddy = np.array(num_samples*[0] + num_samples*[1] + num_samples*[2])

In [ ]:
##GEMINI GENERATED THIS CODE

scaler2 = StandardScaler()
X_train_scaled_buddy = scaler2.fit_transform(X_train_buddy)
X_test_scaled = scaler2.transform(X_test)

from tensorflow import keras
from tensorflow.keras import layers
y_train_buddy_encode = keras.utils.to_categorical(y_train_buddy)  
y_test_encode = keras.utils.to_categorical(y_test)  
model = keras.Sequential([
    layers.Dense(dense_layer_length, activation='relu', input_shape=(4,)), 
    layers.Dense(3, activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled_buddy, y_train_buddy_encode, epochs=100, batch_size=16, verbose=0)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test_encode, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

#Make predictions.
predictions = model.predict(X_test_scaled)
predicted_labels = np.argmax(predictions, axis=1)


In [ ]:
accuracy_buddy = accuracy_score(predicted_labels,y_test )
print(f"\nTest Accuracy: {accuracy_buddy:.4f}")

In [ ]:
Run['accuracy_sop'] = accuracy_sop
Run['accuracy_snn'] = accuracy_snn
Run['accuracy_buddy'] = accuracy_buddy

In [ ]:
pd.DataFrame([Run]).to_sql(table, engine, if_exists = if_exists, index = False)